In [1]:
# Pull price data for listings in the tsx constituent db table using yfinance
# Import packages
import yfinance as yf
import pandas as pd
import numpy as np
import os
from dotenv import load_dotenv

# Import utilities
from etl_utils import extract_query, load_query

In [2]:
load_dotenv()

# Create db connection string
db_username = os.getenv("DB_USERNAME")
db_password = os.getenv("DB_PASSWORD")
db_host = os.getenv("DB_HOST")
db_port = os.getenv("DB_PORT")
db_conn_str = f"mysql+pymysql://{db_username}:{db_password}@{db_host}:{db_port}/stocks_etl_ml_db"

In [3]:
# Import current constituent list
import_query = """
SELECT Company_Symbol
FROM stocks_etl_ml_db.tsx_composite_constituents
WHERE Current_Constituent = 1
"""

constituent_list = extract_query(sql_query=import_query, db_conn_str=db_conn_str)

Connection Successful!
SQL script executed successfully!


In [4]:
# We need to format the tickers to yahoo finance's listing convention for tsx listings
# They end in .to
# Also, for tickers that currently have a "." in the string, it should be replaced with a -
constituent_list["Formatted_Symbol"] = constituent_list["Company_Symbol"].str.replace(".", "-")
constituent_list["Formatted_Symbol"] = constituent_list["Formatted_Symbol"] + ".TO"

In [5]:
aem_ticker_str = constituent_list["Formatted_Symbol"].values[2]

In [6]:
aem_ticker = yf.Ticker(aem_ticker_str)

In [10]:
aem_ticker.get_info()

{'address1': '145 King Street East',
 'address2': 'Suite 400',
 'city': 'Toronto',
 'state': 'ON',
 'zip': 'M5C 2Y7',
 'country': 'Canada',
 'phone': '416 947 1212',
 'fax': '416 367 4681',
 'website': 'https://www.agnicoeagle.com',
 'industry': 'Gold',
 'industryKey': 'gold',
 'industryDisp': 'Gold',
 'sector': 'Basic Materials',
 'sectorKey': 'basic-materials',
 'sectorDisp': 'Basic Materials',
 'longBusinessSummary': "Agnico Eagle Mines Limited, a gold mining company, engages in the exploration, development, and production of precious metals. It explores for gold, silver, zinc, and copper. The company's mines are located in Canada, Australia, Finland, and Mexico; and with exploration and development activities in Canada, Australia, Europe, Latin America, and the United States. Agnico Eagle Mines Limited was incorporated in 1953 and is headquartered in Toronto, Canada.",
 'companyOfficers': [{'maxAge': 1,
   'name': 'Mr. Ammar  Al-Joundi M.B.A., P.Eng.',
   'age': 60,
   'title': 'CE

In [55]:
aem_hist = aem_ticker.history(period="1mo", interval="30m")

In [56]:
aem_hist.reset_index(inplace = True)

In [58]:
aem_hist.head(20)

,Datetime,Open,High,Low,Close,Volume,Dividends,Stock Splits
0,2025-10-27 09:30:00-04:00,218.580002,223.850006,218.309998,222.940002,57839,0.0,0.0
1,2025-10-27 10:00:00-04:00,222.750000,223.139999,215.830002,216.250000,255662,0.0,0.0
2,2025-10-27 10:30:00-04:00,216.580002,216.740005,214.000000,214.889999,310714,0.0,0.0
3,2025-10-27 11:00:00-04:00,214.720001,216.550003,213.009995,216.550003,272772,0.0,0.0
4,2025-10-27 11:30:00-04:00,216.559998,217.000000,214.929993,217.000000,206827,0.0,0.0
5,2025-10-27 12:00:00-04:00,217.000000,217.679993,216.100006,216.779999,225919,0.0,0.0
6,2025-10-27 12:30:00-04:00,216.779999,217.500000,216.580002,217.065002,126150,0.0,0.0
7,2025-10-27 13:00:00-04:00,217.115005,217.910004,217.000000,217.550003,50819,0.0,0.0
8,2025-10-27 13:30:00-04:00,217.554993,218.100006,217.300003,217.479996,69026,0.0,0.0
9,2025-10-27 14:00:00-04:00,217.509995,217.619995,216.500000,216.800003,143635,0.0,0.0
